In [1]:
print("hello")

hello


In [6]:
import dspy

# Use the exact model name you pulled with Ollama (see `ollama list`)
import dspy
lm = dspy.LM("ollama_chat/phi3.5:3.8b-mini-instruct-q4_K_M", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=lm)

In [ ]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "# Returns & Policy" \
"- Perishables (Produce, Seafood, Dairy): 3–7 days." \
"- Beverages unopened: 14 days; opened: no returns." \
"- Non-perishables: 30 days." 
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Announcement of iPhone 14 by Apple Inc. featuring new features highlighted by CEO Tim Cook
['Company', 'Product Announced', 'Key Personnel Mentioned']
[{'entity': 'Apple Inc.', 'type': 'Organization'}, {'entity': 'iPhone 14', 'type': 'Product'}, {'entity': 'Tim Cook', 'type': 'Person'}]


In [ ]:

import re
# Simple SQL Generator signature
class SimpleSQL(dspy.Signature):
    """Generate a SQL query from natural language using simple patterns."""
    
    question = dspy.InputField(desc="The natural language question")
    schema_info = dspy.InputField(desc="Simple description of tables and columns")
    constraints = dspy.InputField(desc="Query constraints and requirements")
    
    sql_query = dspy.OutputField(desc="The SQL query that answers the question")


class TextToSQL(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(SimpleSQL)
        
        # Store common table definitions as a simple string
        self.schema_info = """
        Orders(OrderID, CustomerID, EmployeeID, OrderDate, …)
        "Order Details"(OrderID, ProductID, UnitPrice, Quantity, Discount)
        Products(ProductID, ProductName, SupplierID, CategoryID, UnitPrice, …)
        Customers(CustomerID, CompanyName, Country, …)
        Categories(CategoryID, CategoryName, Description)
        """
        
        # Define common constraints
        self.constraints = """
        - Always use proper SQL syntax with semicolons at the end
        - Use proper JOIN syntax when combining tables
        - Use common retail metrics like revenue, margin, units_sold when appropriate
        - For date ranges, prefer between syntax
        """
    
    def forward(self, question):
        # Identify question type and add relevant constraints
        question_constraints = self.constraints
        
        # Pattern matching to add specific constraints
        if re.search(r'\b(how many|count|total number)\b', question, re.I):
            question_constraints += "\n- Use COUNT() for counting results"
            
        if re.search(r'\b(average|avg)\b', question, re.I):
            question_constraints += "\n- Use AVG() for calculating averages"
            
        if re.search(r'\b(maximum|highest|most expensive|top)\b', question, re.I):
            question_constraints += "\n- Use ORDER BY and LIMIT for finding top values"
        
        if re.search(r'\b(group by|per|each)\b', question, re.I):
            question_constraints += "\n- Use GROUP BY for grouping results"
            q = "duringiji"
            contsraints = {["date":"12-12867-182","dates":"135573651", "category":"Beverages"]}
        

        # Generate the SQL query
        result = self.generator(
            question=question,
            schema_info=self.schema_info,
            constraints=question_constraints
        )
        
        return result.sql_query


# Example usage
if __name__ == "__main__":
    converter = TextToSQL()
    
    # Test with various questions
    questions = [
        "During 'Summer Beverages 1997' as defined in the marketing calendar, which product category had the highest total quantity sold.",
        
    ]
    
    for question in questions:
        sql = converter(question)
        print(f"Question: {question}")
        print(f"SQL: {sql}")
        print("-" * 50)

Question: During 'Summer Beverages 1997' as defined in the marketing calendar, which product category had the highest total quantity sold.
SQL: SELECT c.CategoryName, SUM(od.Quantity) as TotalUnitsSold FROM Orders o JOIN OrderDetails od ON o.OrderID = od.OrderID JOIN Products p ON od.ProductID = p.ProductID JOIN Categories c ON p.CategoryID = c.CategoryID WHERE DATE_FORMAT(o.OrderDate, '%Y') = '1997' AND MONTHNAME(o.OrderDate) IN ('June', 'July', 'August') GROUP BY c.CategoryName ORDER BY TotalUnitsSold DESC LIMIT 1;
--------------------------------------------------


In [44]:
import dspy


lm = dspy.LM("ollama_chat/phi3.5:3.8b-mini-instruct-q4_K_M", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=lm)


class RouterModule(dspy.Signature):
    """
    Classify questions into the appropriate processing method:
    - rag: For questions about policies, documents, guidelines, explanations, or content that would be in text files
    - sql: For questions about structured data, counts, averages, or data that would be in database tables
    - hybrid: For questions requiring both document knowledge and structured data
    """

    question: str = dspy.InputField(desc="The user's question to classify")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning about the classification")
    router = dspy.OutputField(desc="The classification of the question",
        choices=["rag", "sql", "hybrid"])
    confidence: float = dspy.OutputField(desc="Confidence score between 0 and 1")

    @staticmethod
    def _router(question: str):
        classify = dspy.Predict(RouterModule)
        return classify(question=question)

In [45]:
def validate_category(example, prediction, trace=None):
    return prediction.router == example.router

# Create a proper DSPy Module for the router
class RouterProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classify = dspy.Predict(RouterModule)
    
    def forward(self, question):
        return self.classify(question=question)

# Create an instance of the module
router_program = RouterProgram()

# response = router(question="During 'Summer Beverages 1997' as defined in the marketing calendar, which product category had the highest total quantity sold.")
# print(response)

In [46]:
import csv
import json
import dspy
from dspy.evaluate import Evaluate


# Load the trainset
trainset = []
with open('/Users/sreerajnair/my_space/my_space/QnA_Assistant_Using_DSPy_and_LangGraph-main/dataset.json', 'r') as file:
    reader = json.load(file)
    for row in reader:
        example = dspy.Example(question=row['question'], router=row['route']).with_inputs("question") 
        trainset.append(example)

print(f"Loaded {len(trainset)} training examples.")
print(trainset)

# Evaluate our existing function
evaluator = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)
evaluator(router_program, metric=validate_category)

Loaded 6 training examples.
[Example({'question': 'According to the product policy, what is the return window (days) for unopened Beverages? Return an integer.', 'router': 'rag'}) (input_keys={'question'}), Example({'question': "During 'Summer Beverages 1997' as defined in the marketing calendar, which product category had the highest total quantity sold? Return {category:str, quantity:int}.", 'router': 'hybrid'}) (input_keys={'question'}), Example({'question': "Using the AOV definition from the KPI docs, what was the Average Order Value during 'Winter Classics 1997", 'router': 'hybrid'}) (input_keys={'question'}), Example({'question': 'Top 3 products by total revenue all-time. Revenue uses Order Details: SUM(UnitPrice*Quantity*(1-Discount)). Return list[{product:str, revenue:float}].', 'router': 'sql'}) (input_keys={'question'}), Example({'question': "Total revenue from the 'Beverages' category during 'Summer Beverages 1997' dates. Return a float rounded to 2 decimals.", 'router': 'hy

2025/09/22 02:32:27 INFO dspy.evaluate.evaluate: Average Metric: 2 / 6 (33.3%)


,question,example_router,reasoning,pred_router,confidence,validate_category
0,"According to the product policy, what is the return window (days) ...",rag,The question is asking for specific information regarding company ...,rag,0.95,✔️ [True]
1,During 'Summer Beverages 1997' as defined in the marketing calenda...,hybrid,"The question is asking for specific data from an event, likely doc...",sql,0.95,
2,"Using the AOV definition from the KPI docs, what was the Average O...",hybrid,"The question asks about specific financial data, namely ""Average O...",sql,0.95,
3,Top 3 products by total revenue all-time. Revenue uses Order Detai...,sql,The question requires aggregating data from an Order Details table...,sql,0.98,✔️ [True]
4,Total revenue from the 'Beverages' category during 'Summer Beverag...,hybrid,"The question is asking for specific financial data, which typicall...",sql,0.95,


EvaluationResult(score=33.33, results=<list of 6 results>)

In [47]:
# Now use this module with MIPROv2
tp = dspy.MIPROv2(metric=validate_category, auto="light")
optimized_classify = tp.compile(router_program, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)

2025/09/22 02:32:34 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 4

2025/09/22 02:32:34 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/22 02:32:34 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/09/22 02:32:34 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6


100%|██████████| 2/2 [00:00<00:00, 47.13it/s]


Bootstrapped 1 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 3/6


100%|██████████| 2/2 [00:00<00:00, 72.82it/s]


...

2025/09/22 02:36:25 INFO dspy.evaluate.evaluate: Average Metric: 1 / 4 (25.0%)
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 25.0 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 4'].
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 25.0, 75.0, 25.0, 75.0, 50.0, 75.0, 75.0, 25.0]
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 75.0
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: ========================


2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 10 / 10 =====



Average Metric: 3.00 / 4 (75.0%): 100%|██████████| 4/4 [00:00<00:00, 162.88it/s]

2025/09/22 02:36:25 INFO dspy.evaluate.evaluate: Average Metric: 3 / 4 (75.0%)
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 5'].
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 25.0, 75.0, 25.0, 75.0, 50.0, 75.0, 75.0, 25.0, 75.0]
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 75.0
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 11 / 10 =====



Average Metric: 3.00 / 4 (75.0%): 100%|██████████| 4/4 [00:00<00:00, 162.74it/s]

2025/09/22 02:36:25 INFO dspy.evaluate.evaluate: Average Metric: 3 / 4 (75.0%)
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 3'].
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 25.0, 75.0, 25.0, 75.0, 50.0, 75.0, 75.0, 25.0, 75.0, 75.0]
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 75.0
2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/22 02:36:25 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 75.0!
